In [3]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import mean_squared_error, f1_score
from Dashboard.data_preprocessing import load_and_preprocess_data
from Dashboard.model_adapter import ModelAdapter
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score

X_train, X_test, y_train, y_test, task = load_and_preprocess_data("adult")




C:\Users\jfill\PycharmProjects\Masterthesis\venv\lib\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
X_train

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,sex,native-country
37193,0.244808,-0.421287,-0.030373,-0.144804,-0.217127,-0.034087,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States
31093,0.974183,-0.138586,-1.586221,-0.144804,-0.217127,-0.437595,Self-emp-not-inc,10th,Married-civ-spouse,Craft-repair,Husband,White,Male,United-States
33814,-0.338691,-0.160982,-0.419335,-0.144804,-0.217127,-0.034087,Local-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,United-States
14500,-0.776316,-0.604842,-0.419335,-0.144804,-0.217127,-0.034087,Private,HS-grad,Never-married,Adm-clerical,Not-in-family,White,Female,United-States
23399,0.536558,-1.526900,0.358588,-0.144804,-0.217127,0.772930,Private,Assoc-voc,Never-married,Transport-moving,Not-in-family,White,Male,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,-0.192816,4.225724,-0.419335,-0.144804,-0.217127,-0.034087,Private,HS-grad,Married-spouse-absent,Other-service,Not-in-family,Black,Male,United-States
44732,-0.338691,-0.776874,-0.030373,-0.144804,-0.217127,0.772930,Private,Some-college,Married-civ-spouse,Craft-repair,Husband,White,Male,United-States
38158,-0.776316,0.572625,-0.030373,-0.144804,-0.217127,-0.034087,Private,Some-college,Divorced,Exec-managerial,Not-in-family,White,Female,United-States
860,0.536558,-0.880772,0.358588,-0.144804,-0.217127,-0.034087,State-gov,Assoc-voc,Separated,Adm-clerical,Unmarried,Black,Female,United-States


In [5]:
# Encode categorical variables
data_encoded = pd.get_dummies(data, drop_first=True)
#print(data_encoded['sex_Female'].value_counts())
print(data_encoded['sex_Male'].value_counts())
target_encoded = target.map({'>50K': 1, '<=50K': 0})

print(target_encoded)
full_df = data_encoded.copy()
full_df['income'] = target_encoded
empty_rows_count = full_df.isna().sum()
print(empty_rows_count)
full_df.dropna(inplace=True)
print(full_df)

sex_Male
True     32650
False    16192
Name: count, dtype: int64
0        0
1        0
2        1
3        1
4        0
        ..
48837    0
48838    1
48839    0
48840    0
48841    1
Name: class, Length: 48842, dtype: category
Categories (2, int64): [0, 1]
age                               0
fnlwgt                            0
education-num                     0
capital-gain                      0
capital-loss                      0
                                 ..
native-country_Trinadad&Tobago    0
native-country_United-States      0
native-country_Vietnam            0
native-country_Yugoslavia         0
income                            0
Length: 98, dtype: int64
        age    fnlwgt  education-num  capital-gain  capital-loss  \
0      25.0  226802.0            7.0           0.0           0.0   
1      38.0   89814.0            9.0           0.0           0.0   
2      28.0  336951.0           12.0           0.0           0.0   
3      44.0  160323.0           10.0        768

In [49]:
from aif360.datasets import BinaryLabelDataset

# Create a BinaryLabelDataset with the corrected DataFrame
dataset = BinaryLabelDataset(df=full_df,
                             label_names=['income'],
                             protected_attribute_names=['sex_Male'], 
                             favorable_label=1,
                             unfavorable_label=0)


In [50]:
from aif360.metrics import BinaryLabelDatasetMetric

privileged_groups = [{'sex_Male': 1}]
unprivileged_groups = [{'sex_Male': 0}]



metric = BinaryLabelDatasetMetric(dataset, 
                                  unprivileged_groups=unprivileged_groups,
                                  privileged_groups=privileged_groups)

print("Statistical parity difference:", metric.statistical_parity_difference())
print("Disparate impact:", metric.disparate_impact())


Statistical parity difference: -0.19451574596420296
Disparate impact: 0.3596552625800337
